In [1]:
import pyvisa

In [2]:
rm = pyvisa.ResourceManager()

rm.list_resources()

('GPIB0::8::INSTR',)

In [3]:
import srlock

In [13]:
import numpy as np
import srlock

res = "GPIB0::8::INSTR"

with srlock.SR830(resource=res, timeout=10000) as lia:
    print("IDN:", lia.idn())  # IDN?
    lia.outx(1)  # ensure responses to GPIB
    print("Current phase (deg):", lia.phas())  # PHAS?
    lia.fmod(1)  # internal reference
    print("Freq set, read back:", lia.freq())
    lia.slvl(2.4)
    print(lia.slvl())


IDN: Stanford_Research_Systems,SR830,s/n46821,ver1.07
Current phase (deg): 179.96
Freq set, read back: 5.0
2.4


In [11]:
import numpy as np
import srlock
import time

res = "GPIB0::8::INSTR"

with srlock.SR830(resource=res, timeout=10000) as lia:
    print("IDN:", lia.idn())  # IDN?
    lia.outx(1)  # ensure responses to GPIB
    print("Current phase (deg):", lia.phas())  # PHAS?
    lia.fmod(1)  # internal reference
    print("Freq set, read back:", lia.freq())

    # Starting data collection
    freqRange = [i for i in range(5, 100000, 50)]
    voltRange = [i / 10 for i in range(0, 50)]

    # Data collection
    data = np.zeros((len(freqRange), len(voltRange), 4))
    noise = np.zeros((len(freqRange), len(voltRange), 4))

    # Config
    run = 3
    timeConst = 1  # in seconds
    lia.oflt(10)
    lia.sens(21)
    time.sleep(timeConst)

    for i in range(len(freqRange)):
        for j in range(len(voltRange)):
            lia.slvl(voltRange[j])
            lia.freq(freqRange[i])
            # lia.aphs()
            # time.sleep(3*timeConst)
            #lia.agan()
            #time.sleep(3*timeConst)
            print("Frequency:", freqRange[i], "Voltage:", voltRange[j])
            time.sleep(5*timeConst)
            # Poll for values
            runData = np.zeros((run, 4))
            for o in range(run):
                snap_vals = lia.snap([1, 2, 3, 4])
                print(snap_vals)
                runData[o, 0] = snap_vals[0]
                runData[o, 1] = snap_vals[1]
                runData[o, 2] = snap_vals[2]
                runData[o, 3] = snap_vals[3]
                time.sleep(timeConst)
            m = np.mean(runData, axis=0)
            n = np.std(runData, axis=0)
            data[i, j] = m
            noise[i, j] = n

# Save data
np.savez("data.npz", freqRange=freqRange, voltRange=voltRange, data=data, noise=noise)


IDN: Stanford_Research_Systems,SR830,s/n46821,ver1.07
Current phase (deg): -178.91
Freq set, read back: 5.0
Frequency: 5 Voltage: 0.0
[0.00211813, 0.0, 0.00211813, 0.0]
[0.00211908, 0.0, 0.00211908, 0.018]
[0.00211908, 9.53682e-07, 0.00211908, 0.036]
Frequency: 5 Voltage: 0.1
[0.000607494, -3.81473e-06, 0.000607494, -0.312]
[0.00036812, -2.86104e-06, 0.00036812, -0.312]
[0.000227929, -9.53682e-07, 0.000227929, -0.168]
Frequency: 5 Voltage: 0.2
[0.000143052, 0.0, 0.000143052, 0.0]
[0.000151635, 0.0, 0.000151635, 0.0]
[0.000157357, 0.0, 0.000157357, 0.0]


KeyboardInterrupt: 